### 04 - kNN classification
De dataset met alle keypoints en aanvullende informatie zal gebruikt worden om een kNN classificatiemodel te trainen.

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd
import pickle

#### Stap 1. Data splitsen voor upper en lower model

In [28]:
# Keuze voor model maken

train_df = pd.read_csv('keypoints_train_df_YOLOv8m.csv', index_col=0)
test_df = pd.read_csv('keypoints_test_df_YOLOv8m.csv', index_col=0)

#train_df = pd.read_csv('keypoints_train_df_YOLOv8n.csv', index_col=0)
#test_df = pd.read_csv('keypoints_test_df_YOLOv8n.csv', index_col=0)

In [29]:
train_df.head()

,path,pose,pose_code,confidence,x_nose,y_nose,x_left_eye,y_left_eye,x_right_eye,y_right_eye,...,scale_factor,A,B,C,D,E,F,G,H,I
0,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,chin_up,0,0.921941,-0.060636,-0.820229,0.019600,-0.889785,-0.142743,-0.885432,...,6.698616,1.112023,1.113709,0.535651,0.276718,0.303405,1.481170,1.376864,0.579270,0.004591
1,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,chin_up,0,0.922079,-0.298304,-0.801438,-0.214676,-0.869041,-0.335108,-0.866289,...,6.631249,1.055317,1.056769,0.516058,0.270212,0.271870,1.559977,1.290606,0.542095,0.000638
2,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,chin_up,0,0.926764,0.381869,-0.805390,0.398161,-0.860116,0.312483,-0.868173,...,6.670558,1.112931,1.112932,0.512416,0.281943,0.318607,1.319247,1.590114,0.499352,0.006372
3,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,chin_up,0,0.925108,0.012743,-0.615652,0.069975,-0.689205,-0.076766,-0.678715,...,6.523933,1.072322,1.072494,0.515049,0.266445,0.291626,1.240675,1.232930,0.492704,0.009406
4,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,chin_up,0,0.919342,-0.063397,-0.749864,0.024433,-0.822238,-0.152194,-0.813908,...,7.108537,1.365106,1.368935,0.545328,0.464428,0.375341,1.490373,1.483889,0.537466,0.018202


In [57]:
train_upper_df = train_df[train_df['pose_code'] < 8]
test_upper_df = test_df[test_df['pose_code'] < 8]

In [58]:
train_lower_df = train_df[train_df['pose_code'] >= 8]
test_lower_df = test_df[test_df['pose_code'] >= 8]

#### Stap 2. Train- en testdata genereren

In [59]:
# Features die o.b.v. de boxplot analyse interessant lijken worden gekozen
features_upper = ['x_left_shoulder','x_right_shoulder','x_left_elbow', 'x_right_elbow','x_left_wrist', 'x_right_wrist','x_left_hip','x_right_hip','y_nose','y_left_eye','y_right_eye','y_left_shoulder','y_right_shoulder','y_left_elbow','y_right_elbow','y_left_wrist','y_right_wrist','y_left_hip','y_right_hip','c_nose','c_left_eye','c_right_eye','c_left_wrist','c_right_wrist', 'A', 'B', 'C', 'D', 'E', 'F', 'G']

X_train_upper = train_upper_df[features_upper]
y_train_upper = train_upper_df['pose_code']
X_test_upper = test_upper_df[features_upper]
y_test_upper = test_upper_df['pose_code']

In [60]:
# Features die o.b.v. de boxplot analyse interessant lijken worden gekozen
features_lower = ['x_left_hip','x_right_hip','x_left_knee','x_right_knee','x_left_ankle','x_right_ankle','y_left_hip','y_right_hip','y_left_knee','y_right_knee','y_left_ankle','y_right_ankle', 'H', 'I']

X_train_lower = train_lower_df[features_lower]
y_train_lower = train_lower_df['pose_code']
X_test_lower = test_lower_df[features_lower]
y_test_lower = test_lower_df['pose_code']

#### Stap 3. Modellen trainen

TO DO: GridSearchCV, spelen met features, kijken welk model het beter doet
TO DO: o.b.v. eigen logica

In [90]:
knn_upper = KNeighborsClassifier()
knn_upper.set_params(n_neighbors = 5, weights = 'distance')
knn_upper.fit(X_train_upper, y_train_upper)

KNeighborsClassifier(weights='distance')

In [91]:
knn_lower = KNeighborsClassifier()
knn_lower.set_params(n_neighbors = 5, weights = 'distance')
knn_lower.fit(X_train_lower, y_train_lower)

KNeighborsClassifier(weights='distance')

#### Stap 4.a. Upper model evalueren


In [92]:
y_train_upper_pred = knn_upper.predict(X_train_upper)
accuracy_score(y_train_upper, y_train_upper_pred)

1.0

In [93]:
''' Analyse resultaten
Confusion matrix toont voorspellingen op de x-as en de echte waardes op de y-as
- 0: 'chin_up'              --> gaat goed
- 1: 'crossed_arms'         --> gaat goed
- 2: 'delivered_gestures'   --> delivered_gestures wordt te vaak voorspeld, 21 keer hands behinds back wordt gemist bijv.
- 3: 'giving_the_back'      --> gaat perfect
- 4: 'hands_behind_back'    --> gaat goed
- 5: 'hands_in_pockets'     --> ook hier worden hands behinds back vaak gemist
- 6: 'hands_loose'          --> gaat goed
- 7: 'hands_touching_face   --> gaat goed

Conclusie: hands behinds back worden vaker niet gevonden: slechts 61 van de 96
'''

confusion_matrix(y_train_upper, y_train_upper_pred)

array([[ 58,   0,   0,   0,   0,   0,   0,   0],
       [  0, 104,   0,   0,   0,   0,   0,   0],
       [  0,   0, 240,   0,   0,   0,   0,   0],
       [  0,   0,   0,  66,   0,   0,   0,   0],
       [  0,   0,   0,   0,  96,   0,   0,   0],
       [  0,   0,   0,   0,   0, 140,   0,   0],
       [  0,   0,   0,   0,   0,   0,  77,   0],
       [  0,   0,   0,   0,   0,   0,   0, 156]], dtype=int64)

In [94]:
y_test_upper_pred = knn_upper.predict(X_test_upper)
accuracy_score(y_test_upper, y_test_upper_pred)

0.7157360406091371

In [95]:
knn_upper.predict_proba(X_test_upper)

array([[0.        , 0.        , 0.        , ..., 0.3934266 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.75063699, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [96]:
''' Analyse resultaten
Confusion matrix toont voorspellingen op de x-as en de echte waardes op de y-as
- 0: 'chin_up'              --> gaat hier minder goed
- 1: 'crossed_arms'         --> gaat goed
- 2: 'delivered_gestures'   --> gaat hier beter dan bij train
- 3: 'giving_the_back'      --> gaat perfect
- 4: 'hands_behind_back'    --> gaat slecht! wordt te vaak voorspeld
- 5: 'hands_in_pockets'     --> ook hier worden hands behinds back vaak gemist
- 6: 'hands_loose'          --> gaat goed
- 7: 'hands_touching_face   --> gaat goed

Conclusie: hands behinds back worden vaker niet gevonden: slechts 61 van de 96
'''
# To do: mooie heatmap maken

confusion_matrix(y_test_upper, y_test_upper_pred)

array([[11,  0,  1,  0,  2,  1,  0,  0],
       [ 0, 22,  0,  0,  1,  0,  0,  9],
       [ 1,  3, 45,  0,  0,  4,  1,  1],
       [ 0,  0,  0,  6,  0,  1,  0,  1],
       [ 0,  0,  2,  0,  5,  4,  0,  0],
       [ 0,  0,  4,  0,  6, 18,  3,  0],
       [ 5,  0,  0,  0,  1,  3, 10,  0],
       [ 0,  0,  2,  0,  0,  0,  0, 24]], dtype=int64)

#### Stap 4.b. Lower model evalueren


In [97]:
y_train_lower_pred = knn_lower.predict(X_train_lower)
accuracy_score(y_train_lower, y_train_lower_pred)

1.0

In [98]:
confusion_matrix(y_train_lower, y_train_lower_pred)

array([[102,   0,   0,   0],
       [  0,  10,   0,   0],
       [  0,   0,  56,   0],
       [  0,   0,   0, 104]], dtype=int64)

In [99]:
y_test_lower_pred = knn_lower.predict(X_test_lower)
accuracy_score(y_test_lower, y_test_lower_pred)

0.7843137254901961

In [100]:
knn_lower.predict_proba(X_test_lower)

array([[1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [0.80664967, 0.        , 0.19335033, 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [0.37226594, 0.        , 0.44941537, 0.17831869],
       [1.        , 0.        , 0.        , 0.        ],
       [0.80859485, 0.        , 0.19140515, 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        ,

In [101]:
confusion_matrix(y_test_lower, y_test_lower_pred)

array([[16,  0,  1,  0],
       [ 0,  2,  0,  0],
       [ 3,  1, 10,  1],
       [ 1,  0,  4, 12]], dtype=int64)

#### Stap 5. Model en voorspellingen exporteren

In [34]:
# Prediction en proba toevoegen aan DataFrame

train_df['prediction'] = y_train_pred
train_df['proba'] = knn.predict_proba(X_train).max(axis=1)
train_df.head()

,path,pose,pose_code,confidence,x00,y00,x01,y01,x02,y02,...,y16_norm,A,B,C,D,E,F,G,prediction,proba
0,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.914894,0.519160,0.314611,0.534587,0.305057,0.500624,0.305733,...,1.957170,-0.277956,0.288862,0.784445,0.856234,0.717169,0.804296,0.004347,0,1.0
1,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.909827,0.509721,0.315518,0.524141,0.306418,0.489993,0.307169,...,1.988737,0.234085,0.257445,0.700592,0.724724,0.742652,0.801409,0.008705,0,1.0
2,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.906767,0.503642,0.196350,0.515195,0.174214,0.488498,0.178452,...,1.381261,-0.056527,0.083777,0.570295,0.518169,0.876922,0.938895,0.004897,0,1.0
3,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.912517,0.497531,0.165511,0.515113,0.152043,0.480580,0.154027,...,1.987445,-0.476262,0.476776,0.754658,0.754125,0.914735,0.917351,0.011187,0,1.0
4,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.901814,0.542198,0.162231,0.560624,0.147076,0.521831,0.149408,...,1.949706,-0.395487,0.395488,0.753799,0.698847,0.920348,0.917714,0.002302,0,1.0


In [35]:
# DataFrame exporteren
# Keuze maken voor model

train_df.to_csv('train_df_YOLOv8m.csv')
#train_df.to_csv('train_df_YOLOv8n.csv')

In [102]:
# Pickle maken
# Keuze maken voor model

pickle.dump(knn_upper, open('knn_upper_YOLOv8m.pkl', 'wb'))
pickle.dump(knn_lower, open('knn_lower_YOLOv8m.pkl', 'wb'))
#pickle.dump(knn, open('knn_YOLOv8n.pkl', 'wb'))